In [1]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep
from pathlib import Path

In [2]:
def generate_intervals(initial, interval_size):
    intervals = []
    intervals.append((0, initial))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]

        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    return intervals

In [3]:
def build_table(min_pods, max_pods, initial_lat, interval_lat_size, initial_req, interval_req_size):
    requests_intervals = generate_intervals(initial_req, interval_req_size)

    table = []

    for request_interval in requests_intervals:
        latency_intervals = generate_intervals(initial_lat, interval_lat_size)
        for latency_interval in latency_intervals:
            for pod in range(min_pods, max_pods + 1):
                options = np.zeros(max_pods + 1 - min_pods)
                options = [0 for opt in range(len(options))]

                table.append([pod, latency_interval[0], latency_interval[1], request_interval[0], request_interval[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency', 'initial_request', 'end_request']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [4]:
def find_options(table, pod, latency, request):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request)].iloc[:,5::]

In [5]:
def find_best_q_value(table, pod, latency, request):
    print("find_best_q_value")
    options = find_options(table, pod, latency, request)
    
    return int(options.max(axis=1))

In [6]:
def find_best_action(table, pod, latency, request):
    print("find_best_action")
    options = find_options(table, pod, latency, request)
    
    return int(options.idxmax(axis=1))

In [7]:
def update_action_result(table, pod, latency, request, action, result):
    print("update_action_result")
    
    table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request), str(action)] = result
    print("updating pod", pod, " latency ", latency, " request ", request, " result ", result)
    return table

In [8]:
def get_latency():
    print("get_latency")
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[30s])/rate(http_server_request_duration_seconds_count{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return round(float(result) * 1000)

In [44]:
def get_request_by_second():
    print("get_request_by_second")    
    query = 'increase(http_server_requests_total{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
        
    req = round(float(result))
    if req > 54:
        req = 54
    
    return req

In [10]:
def get_pods():
    print("get_pods")    
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [11]:
def set_pods(new_pods):
    print("set_pods")
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [35]:
def run(epochs, table, min_pods, max_pods):
    ideal_latency = 500
    alpha = 0.1
    gamma = 0.6
    epsilon = 0.1
    
    history_file = Path("history.csv")
    if history_file.is_file():
        history = pd.read_csv("history.csv")
    else:
        history = pd.DataFrame([], columns=['epoch','pod', 'latency', 'requests', 'action', 'new_latency', 'new_requests', 'reward', 'q_value', 'random'])      
    
    latency = get_latency()
    pods = get_pods()
    requests = get_request_by_second()

    for x in range(epochs):
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        is_random = None

        if np.random.uniform(0, 1) > epsilon:
            action = find_best_action(table, pods, latency, requests)
            is_random = False
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            is_random = True
            print("random ", action)

        set_pods(action)
        sleep(100)

        new_latency = get_latency()
        new_requests = get_request_by_second()
        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print(" action ", action, "\n new_latency ", new_latency, "\n new_requests ", new_requests, "\n reward ", reward)

        new_value = new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma)
        print("*** new value: ", new_value, " ***")
        
        table = update_action_result(table, pods, latency, requests, action, new_value)
        values = {'epoch': x,'pod': pods, 'latency': latency, 'requests': requests, 'action': action, 'new_latency': new_latency, 'new_requests': new_requests, 'reward': reward, 'q_value' : new_value, 'random': is_random}
        history = history.append(values, ignore_index=True)
        # setting new state
        latency = new_latency
        pods = action
        requests = new_requests
    
    table.to_csv('table.csv', index=False)
    history.to_csv('history.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [13]:
def new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma):
    old_value = find_best_q_value(table, pods, latency, requests)
    next_max = find_best_q_value(table, action, new_latency, new_requests)

    return (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

In [14]:
def is_ideal_latency(ideal_latency, latency):
    print("is_ideal_latency")
    return latency <= ideal_latency

In [15]:
def obtained_result(new_value, old_value):
    print("obtained_result")    
    if new_value == old_value:
        return 'kept'
    elif new_value > old_value:
        return 'increased'
    else:
        return 'decreased'

In [16]:
def reward_within_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_within_ideal")
    if not new_latency_is_ideal:
        return -20 # qualquer ação que tomou resultou em levar a latencia para acima do desejado
    
    if pod_state == 'kept': return 10 # não tomou nenhuma ação, mas manteve a latencia ideal
    elif pod_state == 'increased':
        return {
            'kept': -10, # aumentou pods e a latencia se manteve - pode ter aumentado as requisições
            'increased': 0, # aumentou pods e a latencia aumentou - pode ter aumentado as requisições
            'decreased': -20 # ja estava na latencia ideal e aumentou pods desnecessáriamente
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 20, # diminiu pods e a latencia se manteve
            'increased': 20, # diminiu pods, latencia aumentou mas segue estando dentro do desejado: ótimo cenário
            'decreased': 20 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [17]:
def reward_out_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_out_ideal")    
    if new_latency_is_ideal:
        return 20 # qualquer ação tomada resultou em levar a latencia para dentro do desejado
    
    if pod_state == 'kept':
        return {
            'kept': -20, # não tomou nenhuma ação
            'increased': -20, # não tomou nenhuma ação e aumentou a latencua
            'decreased': -10 # não tomou nenhuma ação mas diminiu a latencia - pode ter diminuido requisições
        }.get(latency_state)
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e manteve a latencia - podem ter aumentado as requisições
            'increased': 0, # aumentou pods e aumentou latencia - podem ter aumentado as requisições
            'decreased': 10 # aumentou pods e diminiu a latencia
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': -10, # diminiu pods e a latencia se manteve
            'increased': -20, # diminiu pods, latencia aumentou
            'decreased': 0 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [18]:
def get_reward(ideal_latency, old_pods, new_pods, old_latency, new_latency):
    print("get_reward")    
    pod_state = obtained_result(new_pods, old_pods)
    latency_state = obtained_result(new_latency, old_latency)

    old_latency_is_ideal = is_ideal_latency(ideal_latency, old_latency)
    new_latency_is_ideal = is_ideal_latency(ideal_latency, new_latency)

    reward = 0
    if old_latency_is_ideal:
        reward = reward_within_ideal(pod_state, latency_state, new_latency_is_ideal)
    else:
        reward = reward_out_ideal(pod_state, latency_state, new_latency_is_ideal)

    return reward

In [36]:
epochs = 100
min_pods = 1
max_pods = 3
initial_latency = 500
interval_size = 4
initial_request = 18
interval_request = 3

In [41]:
# table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)
# table

table_file = Path("table.csv")
if table_file.is_file():
    table = pd.read_csv("table.csv")
else:
    table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)

table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,18,1.96,-0.92,-0.74
1,2,0,500,0,18,-1.40,-1.46,-1.94
2,3,0,500,0,18,-2.30,-1.64,-2.96
3,1,501,1000,0,18,0.00,0.00,0.00
4,2,501,1000,0,18,1.84,-2.00,5.54
5,3,501,1000,0,18,-0.92,0.94,-1.94
6,1,1001,1500,0,18,0.00,0.00,0.00
7,2,1001,1500,0,18,0.00,0.00,0.00
8,3,1001,1500,0,18,0.00,0.00,0.00
9,1,1501,2000,0,18,0.00,0.00,0.00


In [46]:
run(epochs, table, min_pods, max_pods)

get_latency
get_pods
get_request_by_second

--- Exec  0 

 latency  1490 
 pods  1 
 requests  54
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  2 
 new_latency  767 
 new_requests  30 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  5.26  ***
update_action_result
updating pod 1  latency  1490  request  54  result  5.26

--- Exec  1 

 latency  767 
 pods  2 
 requests  30
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  713 
 new_requests  34 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  10.96  ***
update_action_result
updating pod 2  latency  767  request  30  result  10.96

--- Exec  2 

 latency  713 
 pods  3 
 requests  34
find_best_action
choose action  2
set_pods
get_

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  782 
 new_requests  28 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  9.04  ***
update_action_result
updating pod 2  latency  995  request  30  result  9.04

--- Exec  20 

 latency  782 
 pods  3 
 requests  28
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  595 
 new_requests  18 
 reward  -10
find_best_q_value
find_best_q_value
*** new value:  -1.96  ***
update_action_result
updating pod 3  latency  782  request  28  result  -1.96

--- Exec  21 

 latency  595 
 pods  3 
 requests  18
random  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  2 
 new_latency  1063 
 new_re

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  2 
 new_latency  835 
 new_requests  26 
 reward  -20
find_best_q_value
find_best_q_value
*** new value:  5.68  ***
update_action_result
updating pod 2  latency  669  request  24  result  5.68

--- Exec  39 

 latency  835 
 pods  2 
 requests  26
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  800 
 new_requests  32 
 reward  10
find_best_q_value
find_best_q_value
*** new value:  8.08  ***
update_action_result
updating pod 2  latency  835  request  26  result  8.08

--- Exec  40 

 latency  800 
 pods  3 
 requests  32
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  507 
 new_requests  14 
 reward  -10
find_best_q_value
find_best_q_value
*** new value:  -1.96  ***
update_action_result
updating pod 3  latency  673  request  18  result  -1.96

--- Exec  58 

 latency  507 
 pods  3 
 requests  14
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  587 
 new_requests  18 
 reward  -20
find_best_q_value
find_best_q_value
*** new value:  -2.96  ***
update_action_result
updating pod 3  latency  507  request  14  result  -2.96

--- Exec  59 

 latency  587 
 pods  3 
 requests  18
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  1 


get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  425 
 new_requests  12 
 reward  20
find_best_q_value
find_best_q_value
*** new value:  9.08  ***
update_action_result
updating pod 2  latency  561  request  24  result  9.08

--- Exec  78 

 latency  425 
 pods  3 
 requests  12
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  746 
 new_requests  24 
 reward  -20
find_best_q_value
find_best_q_value
*** new value:  -3.2600000000000002  ***
update_action_result
updating pod 3  latency  425  request  12  result  -3.2600000000000002

--- Exec  79 

 latency  746 
 pods  2 
 requests  24
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
r

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  631 
 new_requests  22 
 reward  -10
find_best_q_value
find_best_q_value
*** new value:  -2.92  ***
update_action_result
updating pod 3  latency  652  request  32  result  -2.92

--- Exec  97 

 latency  631 
 pods  3 
 requests  22
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  593 
 new_requests  20 
 reward  -10
find_best_q_value
find_best_q_value
*** new value:  -2.92  ***
update_action_result
updating pod 3  latency  631  request  22  result  -2.92

--- Exec  98 

 latency  593 
 pods  3 
 requests  20
find_best_action
choose action  3
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 


In [47]:
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,18,1.96,-0.92,-0.74
1,2,0,500,0,18,-1.40,-1.46,-1.94
2,3,0,500,0,18,-3.74,-4.34,-3.80
3,1,501,1000,0,18,0.00,0.00,0.00
4,2,501,1000,0,18,1.84,-2.00,6.38
5,3,501,1000,0,18,-3.80,-4.28,-3.92
6,1,1001,1500,0,18,0.00,0.00,0.00
7,2,1001,1500,0,18,0.00,0.00,0.00
8,3,1001,1500,0,18,0.00,0.00,0.00
9,1,1501,2000,0,18,0.00,0.00,0.00


In [26]:
import datetime

In [33]:
print("acabou: ", datetime.datetime.now())

acabou:  2021-10-24 00:21:57.819061


In [45]:
find_best_q_value(table, 1, 500, 56)

find_best_q_value


TypeError: cannot convert the series to <class 'int'>